In [276]:
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range #################################################
import numpy as np
from dataclasses import dataclass
import astropy.units as u
from astropy.coordinates import SkyCoord, SkyOffsetFrame
from datetime import datetime
from numba import njit



@njit
def calculate_distance_matrix(quick_list, angle_weight, time_weight, max_distance):
    l = len(quick_list)
    distances = np.full((l,l), 2*max_distance)
    
    partitions = [0]
    for i in range(1,l):
        if quick_list[i,2]-quick_list[i-1,2] > max_distance/time_weight:
            partitions.append(i)
    partitions.append(l)
    
    for i in range(len(partitions)-1):
        for j in range(partitions[i], partitions[i+1]):

            for k in range(j+1, partitions[i+1]):
                distances[j,k] = distances[k,j] = calculate_distance(quick_list[j],quick_list[k],
                                                                     angle_weight,time_weight)
                
    np.fill_diagonal(distances,0.)
            
    return np.array(partitions), distances

@njit
def calculate_distance(point1, point2, angle_weight, time_weight): #include minimum distance ################################################
    ang_dis = np.arccos( np.sin(point1[1])*np.sin(point2[1]) 
                        + np.cos(point1[1])*np.cos(point2[1]) * np.cos(point1[0] - point2[0]) )
    if np.isnan(ang_dis): ############ find better solution?
        print("ANG_DIS IS NAN!")
    time_dis = abs(point1[2] - point2[2])
    return ( (angle_weight*ang_dis)**2 + (time_weight*time_dis)**2 )**0.5

@njit
def find_regions(distances, max_distance, partitions):
    regions = []
    for i,partition in enumerate(partitions[:-1]):
        unconnected = [j for j in range(partition, partitions[i+1])]
        while not len(unconnected)==0:
            temp_region = [unconnected.pop(0)]
            search_index = 0
            while search_index < len(temp_region):
                l = len(unconnected)
                for j in range(l-1,-1,-1):
                    if distances[ temp_region[search_index], unconnected[j] ] < max_distance:
                        temp_region.append(unconnected.pop(j))
                search_index += 1
            regions.append(sorted(temp_region))
    return regions

@njit
def choose_random_weighted_interval(weights):
    r = np.random.random(1)[0] * np.sum(weights)
    s = 0.
    for i, w in enumerate(weights):
        s += w
        if r < s:
            return i
        
@njit
def calc_pair_combinations(number):
    return (number)*(number-1)/2
    
    

class Cluster:
    def __init__(self,
                 pointing,
                 region):
        self.indices = [pointing.index]
        self.avg_distance = 0.
        self.num_pointings = 1
        self.pointings = [pointing]
        self.region = region
        
    def add_pointing(self, pointing):
        self.avg_distance = self.calc_new_avg_dist(pointing)
        self.indices.append(pointing.index)
        self.pointings.append(pointing)
        self.num_pointings += 1
        
    def should_add_pointing(self, pointing):
        if self.num_pointings < self.region.query._cluster_size_range[1]:
            if self.find_new_max_dist(pointing) < self.region.query._max_distance:
                if self.num_pointings >= self.region.query._cluster_size_range[0]:
                    if (self.calc_new_avg_dist(pointing)/self.avg_distance ################## make sigmoid? why make sigmoid?
                        < self.region.query._cluster_size_preference_threshold[self.num_pointings 
                                                                                - self.region.query._cluster_size_range[0]]):
                        return True
                else:
                    return True
        return False
    
        
    def dissolve_cluster(self): ################################
        for p in self.pointings:
            p.cluster = None
            
    def finalize_cluster(self): #######################################
        for p in self.pointings:
            p.cluster = self
            
        
        
    def calc_new_avg_dist(self, pointing):
        return ((self.avg_distance * calc_pair_combinations(self.num_pointings)
                 + np.sum(self.region.query._distances[pointing.index,self.indices]))
                 / calc_pair_combinations(self.num_pointings+1) )
        
    def find_new_max_dist(self, pointing): # only checks for new pointing!
        return np.amax(self.region.query._distances[pointing.index,self.indices])

    def find_closest_pointings(self, cluster2):
        d = self.region.query._distances[self.indices,:][:,cluster2.indices]
        c = np.unravel_index(d.argmin(), d.shape)
        return self.pointings[c[0]], cluster2.pointings[c[1]]
    
        

@dataclass
class Pointing:
    '''
    Dataclass that represents a single Pointing
    '''
    scw_id: str
    sky_coords: SkyCoord
    start_time: datetime
    index: int
    cluster: Cluster = None
    # delete unnecessary clusters and regions ##############################################
    
    def distance_calculator(self, pointing2, angle_weight: float, time_weight: float): #####################################################
        return ( (angle_weight * self.sky_coords.separation(pointing2.sky_coords).deg)**2
                + (time_weight * abs( (self.start_time - pointing2.start_time).total_seconds()/86400 ) )**2 )**0.5
        
    def angle_between_three_pointings(self, pointing2, pointing3, angle_weight, time_weight):
        origin = SkyCoord(self.sky_coords.ra.deg, (self.sky_coords.dec.deg%180.)-90., frame="icrs",unit="deg")
        origin_Frame = SkyOffsetFrame(origin = origin)
        
        p1 = self.sky_coords.transform_to(origin_Frame)
        p2 = pointing2.sky_coords.transform_to(origin_Frame)
        p3 = pointing3.sky_coords.transform_to(origin_Frame)
        
        a_a = p2.lon.deg - p3.lon.deg
        a_d2 = abs(p1.lat.deg - p2.lat.deg) * angle_weight
        a_d3 = abs(p1.lat.deg - p3.lat.deg) * angle_weight
        
        t_d2 = (pointing2.start_time - self.start_time).total_seconds()/86400 * time_weight
        t_d3 = (pointing3.start_time - self.start_time).total_seconds()/86400 * time_weight
        
        return np.arccos( np.clip((a_d2*a_d3*np.cos(np.deg2rad(a_a)) + t_d2*t_d3) 
                                  / np.linalg.norm([a_d2,t_d2]) / np.linalg.norm([a_d3,t_d3]), -1., 1.) )
        
        





class ClusteredQuery:
    def __init__(self,
                 scw_ids, # Has to be sorted by START_DATE
                 angle_weight,
                 time_weight,
                 max_distance,
                 cluster_size_range = (3,5),
                 cluster_size_preference_threshold = (3,3),
                 failed_improvements_max = 4,
                 suboptimal_cluster_size_range = (1,2), #has to start at 1
                 close_suboptimal_cluster_size_range = (1,3) # above has to be subset
                 ):
        """
        Init the Clustered Query object. Used to cluster pointings
        
        """
        self._angle_weight = float(angle_weight)
        self._time_weight = float(time_weight)
        self._max_distance = float(max_distance)
        self._cluster_size_range = cluster_size_range
        self._cluster_size_preference_threshold = cluster_size_preference_threshold
        self._failed_improvements_max = failed_improvements_max
        self._suboptimal_cluster_size_range = suboptimal_cluster_size_range
        self._close_suboptimal_cluster_size_range = close_suboptimal_cluster_size_range
        
        self._num_pointings = len(scw_ids)
        
        self.quick_list = np.zeros((self._num_pointings, 3))
        self.quick_list[:,0:2] = scw_ids[:,1:3]
        for i in range(self._num_pointings):
            self.quick_list[i,2] = (scw_ids[i,3] - datetime(2000,1,1,0,0,0)).total_seconds()/86400
            
        partitions, self._distances = calculate_distance_matrix(self.quick_list, angle_weight, time_weight, self._max_distance)
        
        self._sortable_distances = np.concatenate((self.distances,np.array([np.arange(self._num_pointings)]).T),axis=1) ################# point?
        
        self._region_indices = find_regions(self._distances, self._max_distance, partitions)
        
        
        self._pointings = np.array([Pointing(pointing[0],
                                            SkyCoord(pointing[1],pointing[2],frame="icrs",unit="deg"),
                                            pointing[3], index)
                                    for index, pointing in enumerate(scw_ids)])
                
        self._regions = []
        
        for i in self._region_indices:
            self._regions.append(Region(i, self))
            
    
    
    @property
    def pointings(self):
        """
        :returns: Base List of Pointings
        """
        return self._pointings
    
    @property
    def distances(self):
        """
        :returns: Base Distance Matrix of Pointings
        """
        return self._distances
    

    
    
class Region:
    def __init__(self,
                 region_indices,
                 query
                 ):
        self.indices = region_indices
        
        self.query = query
        
        self.distances = query._distances[self.indices,:][:,self.indices] ############ point?
        self.sortable_distances = np.concatenate((self.distances,np.array([self.indices]).T),axis=1) # Adds indices as right column, sort using a[a[:,x].argsort()]
        
        self.clusters = {}
        self.potential_clusters1 = {}
        self.potential_clusters2 = {}
        for i in range(query._cluster_size_range[1]):
            self.clusters[i+1]=[]
            self.potential_clusters1[i+1]=[]
            self.potential_clusters2[i+1]=[]
            

        
        self.initial_clustering()
        
        for key, value in self.clusters.items():
            for c in value:
                print(f"{c.indices}, {c.avg_distance}")
        print(self.has_suboptimal_clusters())
        
        self.attempt_improvement()
        
        # failed_improvements = 0
        # while failed_improvements < self.query._failed_improvements_max and self.has_suboptimal_clusters():
        #     if not self.attempt_improvement():
        #         failed_improvements += 1
        #     else:
        #         failed_improvements = 0
        
    
    def initial_clustering(self): ################## also check following pointings
        cluster = Cluster(self.query._pointings[self.indices[0]], self)
        for index in self.indices[1:]:
            if cluster.should_add_pointing(self.query._pointings[index]):
                cluster.add_pointing(self.query._pointings[index])
            else:
                cluster.finalize_cluster()
                self.clusters[cluster.num_pointings].append(cluster)
                cluster = Cluster(self.query._pointings[index], self)
        cluster.finalize_cluster()
        self.clusters[cluster.num_pointings].append(cluster)
        
    def attempt_improvement(self):
        c1 = self.find_suboptimal_cluster()
        c2 = self.find_close_suboptimal_cluster(c1)
        found_path, recluster_indices = self.find_cluster_path(c1,c2)
        if not found_path:
            return False
        else:
            self.recluster_pointings(recluster_indices, c1)
            pass
            
        
    
    def find_suboptimal_cluster(self):
        size_weights = np.array([len(self.clusters[i]) / i**2
                                 for i in range(self.query._suboptimal_cluster_size_range[0],
                                                self.query._suboptimal_cluster_size_range[1] + 1)])
        size = choose_random_weighted_interval(size_weights) + 1
        index = np.random.randint(len(self.clusters[size]))
        cluster = self.clusters[size].pop(index)
        self.potential_clusters1[size].append(cluster)
        return cluster
    
    def find_close_suboptimal_cluster(self, cluster):
        clusters = []
        cluster_size_indices = [0]
        for i in range(self.query._close_suboptimal_cluster_size_range[0], 
                       self.query._close_suboptimal_cluster_size_range[1] + 1):
            clusters.extend(self.clusters[i])
            cluster_size_indices.append( len(self.clusters[i]) + cluster_size_indices[i-1] )
        cluster_weights = np.zeros(len(clusters))
        for i, c in enumerate(clusters):
            d = self.query._distances[cluster.indices,:][:,c.indices]
            cluster_weights[i] = np.exp( -2.*np.amin(d) / self.query._max_distance )
        for i in range(1, self.query._close_suboptimal_cluster_size_range[1] + 1):
            cluster_weights[cluster_size_indices[i-1]:cluster_size_indices[i]] /= i
        index = choose_random_weighted_interval( cluster_weights )
        for size, csi in enumerate(cluster_size_indices):
            if not index >= csi:
                break
        true_index = index - cluster_size_indices[size-1]
        
        cluster2 = self.clusters[size].pop(true_index)
        self.potential_clusters1[size].append(cluster2)
        return cluster2
        
        
        
        
    
    def find_cluster_path(self, cluster1, cluster2):
        indices_in = set(cluster1.indices)
        indices_to = set(cluster2.indices)
        arrived = False
        while not arrived:
        
            indices_out = np.array([i for i in self.indices if i not in indices_in])
            pointing1, pointing2 = cluster1.find_closest_pointings(cluster2)
            
            num_points = min(self.query._cluster_size_range[1]*4, len(indices_out))

            sortable_pointings = [i for i in range(num_points)]
            distances = self.query._distances[pointing1.index,indices_out]
            close_indices = indices_out[np.argpartition(distances, sortable_pointings)[:num_points]]
            
            angle = np.vectorize(lambda p: pointing1.angle_between_three_pointings(pointing2, p, self.query._angle_weight,
                                                                                   self.query._time_weight))
            
            angles_filtered = angle(self.query._pointings[close_indices])
            
            distances_filtered = self.query._distances[pointing1.index, close_indices]
            
            close_indices_weights = ((distances_filtered < self.query._max_distance) 
                                     * np.exp(-3. * distances_filtered / self.query._max_distance) 
                                     * np.cos(angles_filtered/2.)**4)
            
            if np.amax(np.cos(angles_filtered)) < 0.:
                print("Cannot go forwards!")
                return False, None
            
            
            index = close_indices[choose_random_weighted_interval(close_indices_weights)]
            if index is None:
                return False, None
            cluster1 = self.query._pointings[index].cluster
            indices_in = indices_in | set(cluster1.indices)
            if index in indices_to:
                arrived = True
            else:
                self.clusters[cluster1.num_pointings].remove(cluster1)
                self.potential_clusters1[cluster1.num_pointings].append(cluster1)
        
        return True, list(indices_in)
        
    def recluster_pointings(self, recluster_indices, start_cluster):
        remove = recluster_indices.remove
        cluster = start_cluster
        while recluster_indices:
            pass
        
        
        start_index = start_cluster.indices[choose_random_weighted_interval( 
                                     np.exp(np.average(self.query._distances[c1.indices,:][:,recluster_indices], axis=1)
                                            / self.query._max_distance) )]
        cluster = Cluster(self.query._pointings[start_index], self)
        remove = recluster_indices.remove
        remove(start_index)
        ####################################################################################### make non recursive
    
    def calc_cluster_cost(self, clusters):
        pass
    
    def has_suboptimal_clusters(self):
        s = 0
        for i in range(self.query._suboptimal_cluster_size_range[0], 
                       self.query._suboptimal_cluster_size_range[1] + 1):
            s += len(self.clusters[i])
        if s >= 2:
            return True
        elif s >= 1:
            for i in range(self.query._suboptimal_cluster_size_range[1] + 1,
                           self.query._close_suboptimal_cluster_size_range[1] + 1):
                s += len(self.clusters[i])
            if s >= 2:
                return True
        return False
            
        






In [277]:
searchquerry = SearchQuery(object_name="Cyg X-1", resultmax=0)
cat = IntegralQuery(searchquerry)
f = Filter(SCW_TYPE="POINTING")
scw_ids = cat.apply_filter(f,True)

In [278]:
test = ClusteredQuery(scw_ids[:4], 1, 1, 2.8, cluster_size_preference_threshold = (1,1))

[3], 0.0
[0, 1, 2], 0.08185850791460011
True
3 0
{3}
[0 1 2]
{0, 1, 2}
[1.18375462 1.2789768  1.27787504]
[0 2 1]
Pointing(scw_id='001130000100', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (300.902863, 36.89711)>, start_time=datetime.datetime(2002, 11, 16, 21, 55, 17), index=3, cluster=<__main__.Cluster object at 0x7f6480f2d400>)
Pointing(scw_id='001130000050', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (299.543488, 35.301971)>, start_time=datetime.datetime(2002, 11, 16, 20, 6, 11), index=0, cluster=<__main__.Cluster object at 0x7f6481e48d60>)
Pointing(scw_id='001130000050', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (299.543488, 35.301971)>, start_time=datetime.datetime(2002, 11, 16, 20, 6, 11), index=0, cluster=<__main__.Cluster object at 0x7f6481e48d60>)
<SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (300.902863, -53.10289)>): (lon, lat) in deg
    (0., 90.)> <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coor

In [279]:
test = ClusteredQuery(scw_ids[:20], 1, 1, 2.8, cluster_size_preference_threshold = (100,100))
#print(test._region_indices)

[19], 0.0
[5, 6, 7], 1.831296171661494
[12, 13, 14], 1.8267541396306226
[8, 9, 10, 11], 1.2999530762680331
[15, 16, 17, 18], 1.2981536904708968
[0, 1, 2, 3, 4], 0.7751899214493297
True
19 12
{19}
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
{12, 13, 14}
[2.13134336 2.02585686 2.02037569 2.83672194 2.83398732 0.32954735
 0.94084485 2.6388627  1.9844859  1.99202434 2.82089881 1.96970354
 0.17564257 0.89737088 2.61885778 1.97031682 1.97810561 2.81000306
 1.96682056]
[12  5 13  6 18 11 15 16  8  9  2  1  0 14  7 17 10  4  3]
Pointing(scw_id='001130000410', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (300.735809, 33.439304)>, start_time=datetime.datetime(2002, 11, 17, 6, 52, 58), index=19, cluster=<__main__.Cluster object at 0x7f644317c520>)
Pointing(scw_id='001130000270', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (300.728149, 33.435944)>, start_time=datetime.datetime(2002, 11, 17, 2, 40, 8), index=12, cluster=<__main__.Cluster object at 0x7f644317c790>)
Pointing

In [280]:
test._sortable_distances

array([[ 0.        ,  0.10729474,  0.11235486,  1.18375462,  1.18531483,
         2.08635343,  2.06212579,  1.58002267,  1.34863689,  0.23371449,
         1.20341017,  1.72466543,  2.10084478,  2.07790924,  1.6137347 ,
         1.3772865 ,  0.39811707,  1.2552358 ,  1.75228592,  2.13134336,
         0.        ],
       [ 0.10729474,  0.        ,  0.02592593,  1.2789768 ,  1.28027953,
         1.97984266,  1.99316623,  1.6834768 ,  1.39503412,  0.19934031,
         1.29619747,  1.78866378,  1.99445148,  2.00837624,  1.71454867,
         1.42134105,  0.37491915,  1.34383332,  1.81456626,  2.02585686,
         1.        ],
       [ 0.11235486,  0.02592593,  0.        ,  1.27787504,  1.27874555,
         1.97855624,  1.99160367,  1.6812853 ,  1.39197574,  0.17341439,
         1.29202515,  1.78532322,  1.99117119,  2.00483602,  1.70973693,
         1.41512697,  0.34899323,  1.33640774,  1.80875855,  2.02037569,
         2.        ],
       [ 1.18375462,  1.2789768 ,  1.27787504,  0.        

In [295]:
a = np.reshape( np.arange(100), (10,10) )
b = a[[1,4,5],:][:,[3,7,8,9]]
a, b

(array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
        [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]]),
 array([[13, 17, 18, 19],
        [43, 47, 48, 49],
        [53, 57, 58, 59]]))

In [301]:
np.sum(b, axis=1)


array([ 67, 187, 227])

In [283]:
c = np.reshape( np.random.randint(1000, size=100), (10,10) )
c = np.concatenate((c,np.array([np.arange(10)]).T),axis=1)
d = (0,1,2,3,4)
c

array([[839, 964, 924, 113, 727, 960, 354, 523, 679, 595,   0],
       [ 48, 646, 999, 762, 592, 104, 461, 906, 202, 127,   1],
       [502, 904, 653, 610, 642, 267, 964, 744, 171, 809,   2],
       [ 83, 194,   7, 896, 363, 167, 498, 338, 293, 643,   3],
       [563, 386, 394, 963, 866, 932, 616, 959, 720, 196,   4],
       [225, 111, 674, 727, 344, 556, 931, 914, 146, 172,   5],
       [246, 373, 315,  85,  54,  27, 736, 524, 226, 452,   6],
       [964,  11, 822, 980,  98, 507,  43, 170,  48, 760,   7],
       [555, 839, 844,  41,  84, 601, 167, 934, 790, 795,   8],
       [650, 183, 680, 567, 117, 883, 141, 260, 895, 555,   9]])

In [284]:
c[np.argpartition(c[:,0], d)]

array([[ 48, 646, 999, 762, 592, 104, 461, 906, 202, 127,   1],
       [ 83, 194,   7, 896, 363, 167, 498, 338, 293, 643,   3],
       [225, 111, 674, 727, 344, 556, 931, 914, 146, 172,   5],
       [246, 373, 315,  85,  54,  27, 736, 524, 226, 452,   6],
       [502, 904, 653, 610, 642, 267, 964, 744, 171, 809,   2],
       [563, 386, 394, 963, 866, 932, 616, 959, 720, 196,   4],
       [839, 964, 924, 113, 727, 960, 354, 523, 679, 595,   0],
       [964,  11, 822, 980,  98, 507,  43, 170,  48, 760,   7],
       [555, 839, 844,  41,  84, 601, 167, 934, 790, 795,   8],
       [650, 183, 680, 567, 117, 883, 141, 260, 895, 555,   9]])

In [285]:
lon, lat = 300., -0.1

o = SkyCoord(lon,(lat%180.)-90.,frame="icrs",unit="deg")
oF = SkyOffsetFrame(origin = o)
a = SkyCoord(lon,lat,frame="icrs",unit="deg")
b = SkyCoord(lon-.1,lat+.1,frame="icrs",unit="deg")
c = SkyCoord(lon+.1,lat-.1,frame="icrs",unit="deg")
aT, bT, cT = a.transform_to(oF), b.transform_to(oF), c.transform_to(oF)
aT, bT, cT



(<SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
     (300., 89.9)>): (lon, lat) in deg
     (-180., -90.)>,
 <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
     (300., 89.9)>): (lon, lat) in deg
     (-45.00004363, -89.85857868)>,
 <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
     (300., 89.9)>): (lon, lat) in deg
     (135.00021817, -89.8585789)>)

In [286]:
(100%180.)-90

10.0

In [287]:
test.pointings[19], test.pointings[12], test.pointings[5]


(Pointing(scw_id='001130000410', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
     (300.735809, 33.439304)>, start_time=datetime.datetime(2002, 11, 17, 6, 52, 58), index=19, cluster=<__main__.Cluster object at 0x7f644317c520>),
 Pointing(scw_id='001130000270', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
     (300.728149, 33.435944)>, start_time=datetime.datetime(2002, 11, 17, 2, 40, 8), index=12, cluster=<__main__.Cluster object at 0x7f644317c790>),
 Pointing(scw_id='001130000130', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
     (300.728149, 33.436001)>, start_time=datetime.datetime(2002, 11, 16, 22, 58, 28), index=5, cluster=<__main__.Cluster object at 0x7f644317c880>))

In [294]:
test.pointings[17].angle_between_three_pointings(test.pointings[17],test.pointings[7], 1, 1)

Pointing(scw_id='001130000370', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (300.894806, 36.900806)>, start_time=datetime.datetime(2002, 11, 17, 5, 49, 46), index=17, cluster=<__main__.Cluster object at 0x7f644317c820>)
Pointing(scw_id='001130000370', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (300.894806, 36.900806)>, start_time=datetime.datetime(2002, 11, 17, 5, 49, 46), index=17, cluster=<__main__.Cluster object at 0x7f644317c820>)
Pointing(scw_id='001130000170', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (297.14682, 35.230167)>, start_time=datetime.datetime(2002, 11, 17, 0, 1, 50), index=7, cluster=<__main__.Cluster object at 0x7f644317c880>)
<SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (300.894806, -53.099194)>): (lon, lat) in deg
    (-180., 90.)> <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (300.894806, -53.099194)>): (lon, lat) in deg
    (-180., 90.)> <SkyCoord (SkyOff

/tmp/ipykernel_348/800507057.py:173: RuntimeWarning: invalid value encountered in double_scalars
  print(np.clip((a_d2*a_d3*np.cos(np.deg2rad(a_a)) + t_d2*t_d3)
/tmp/ipykernel_348/800507057.py:176: RuntimeWarning: invalid value encountered in double_scalars
  return np.arccos( np.clip((a_d2*a_d3*np.cos(np.deg2rad(a_a)) + t_d2*t_d3)


nan

In [289]:
np.arccos( (2.106609811954499 * 0.10773400734085214 *np.cos(np.deg2rad(7.194815006054043)) + 0.11964120370370371 * 0.2078125) / (2.106609811954499**2 + 0.11964120370370371**2)**0.5 / (0.10773400734085214**2 + 0.2078125**2)**0.5)

1.0399904350287366

In [290]:
a = 3.4593469627040747 * 0.24162037037037037 * np.cos(np.deg2rad(0.)) + 3.4593469627040747 * 0.24162037037037037
b = np.linalg.norm([3.4593469627040747 , 3.4593469627040747 ])
c = np.linalg.norm([-0.24162037037037037 , -0.24162037037037037 ])
a, b, c, b*c, a/b/c, np.arccos(a/b/c)

(1.6716973887363487,
 4.892255391610276,
 0.3417028047233881,
 1.6716973887363489,
 1.0,
 0.0)

In [291]:
a = np.random.random(10)
b = np.random.random(10)
a,b

(array([0.56326099, 0.1944985 , 0.47679831, 0.79751122, 0.49144697,
        0.04228994, 0.83784117, 0.78938221, 0.98923317, 0.3322846 ]),
 array([0.72079036, 0.75069816, 0.77570293, 0.44926497, 0.95825766,
        0.81675017, 0.09676308, 0.38400854, 0.75648149, 0.65930325]))

In [292]:
(a>0.5)*a

array([0.56326099, 0.        , 0.        , 0.79751122, 0.        ,
       0.        , 0.83784117, 0.78938221, 0.98923317, 0.        ])

In [293]:
pointing1 = test.pointings[1]
pointing2 = test.pointings[0]
pointing3 = test.pointings[2]
angle = lambda p : pointing1.angle_between_three_pointings(pointing2, p, 1, 1)
angle2 = np.vectorize(angle)
a = map(angle, test.pointings) 
angle2(test.pointings)


Pointing(scw_id='001130000070', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (299.590179, 35.201195)>, start_time=datetime.datetime(2002, 11, 16, 20, 18, 23), index=1, cluster=<__main__.Cluster object at 0x7f6443819fa0>)
Pointing(scw_id='001130000050', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (299.543488, 35.301971)>, start_time=datetime.datetime(2002, 11, 16, 20, 6, 11), index=0, cluster=<__main__.Cluster object at 0x7f6443819fa0>)
Pointing(scw_id='001130000050', sky_coords=<SkyCoord (ICRS): (ra, dec) in deg
    (299.543488, 35.301971)>, start_time=datetime.datetime(2002, 11, 16, 20, 6, 11), index=0, cluster=<__main__.Cluster object at 0x7f6443819fa0>)
<SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (299.590179, -54.798805)>): (lon, lat) in deg
    (0., 90.)> <SkyCoord (SkyOffsetICRS: rotation=0.0 deg, origin=<ICRS Coordinate: (ra, dec) in deg
    (299.590179, -54.798805)>): (lon, lat) in deg
    (-159.28910213, 89.89225202)> <Sk

/tmp/ipykernel_348/800507057.py:173: RuntimeWarning: invalid value encountered in double_scalars
  print(np.clip((a_d2*a_d3*np.cos(np.deg2rad(a_a)) + t_d2*t_d3)
/tmp/ipykernel_348/800507057.py:176: RuntimeWarning: invalid value encountered in double_scalars
  return np.arccos( np.clip((a_d2*a_d3*np.cos(np.deg2rad(a_a)) + t_d2*t_d3)


array([1.49011612e-08,            nan, 1.64926486e+00, 9.20069487e-01,
       9.21437060e-01, 3.00651318e+00, 2.23022654e+00, 1.19154731e+00,
       2.13372968e-01, 1.64919447e+00, 9.31732950e-01, 1.96719153e+00,
       2.99862135e+00, 2.23535444e+00, 1.20696173e+00, 2.87448603e-01,
       1.65012943e+00, 9.46254732e-01, 1.96486275e+00, 2.94795014e+00])

In [302]:
a = [1,2,3]
remove = a.remove
remove(2)
a

[1, 3]